# Universal Automated Tutor

## 0: Data Import + Cleaning

In [1]:
import pandas as pd
import numpy as np 
import random

In [2]:
# Read in the CSVs, select variables, combine
raw_data04 = pd.read_csv("2004-WPI-Assistments-Math.csv",low_memory=False)
raw_data056 = pd.read_csv("2005-06-WPI-Assistments-Math.csv",low_memory=False)
raw_data = pd.concat([raw_data056,raw_data04])
data = raw_data[["stud_id","duration","student_response_type","problem_id","step","attempt_num",
             "last_attempt","outcome","input","feedback"]]
data.reset_index(drop=True, inplace=True)

# Create dummy variables for hint, correct, incorrect
data.loc[:,"hint"] = np.where(data["outcome"] == "HINT",1,0)
data.loc[:,"correct"] = np.where(data["outcome"] == "CORRECT",1,0)
data.loc[:,"incorrect"] = np.where(data["outcome"] == "INCORRECT",1,0)

# Filter + clean data
data = data[data["duration"] != '.']
data["duration"] = data["duration"].astype(np.float64)

C:\Users\C22Evan.Marrone\Anaconda3\lib\site-packages\pandas\core\indexing.py:1597: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value
C:\Users\C22Evan.Marrone\Anaconda3\lib\site-packages\pandas\core\indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


## 1: Distribution Creation

In [3]:
def chi2lower(x):
    '''
    Calculates the chi-squared distribution lower bound
    '''
    return np.mean(x) - np.std(x)/4


def chi2upper(x):
    '''
    Calculates the chi-squared distribution upper bound
    '''
    return np.mean(x) + np.std(x)/2

In [4]:
# manipulate to make problem distributions
df = data[["problem_id", "duration", "hint", "stud_id","incorrect"]].groupby(["problem_id","stud_id"]).sum()
problemDistsLower = df.groupby(["problem_id"]).agg(chi2lower)
problemDistsUpper = df.groupby(["problem_id"]).agg(chi2upper)
problemDistsLower.rename(columns={'duration': 'durationLower', 'hint': 'hintLower', 'incorrect': 'incorrectLower'}, inplace=True)
problemDistsUpper.rename(columns={'duration': 'durationUpper', 'hint': 'hintUpper', 'incorrect': 'incorrectUpper'}, inplace=True)
problemDists = pd.concat([problemDistsLower, problemDistsUpper], axis=1)
                                           
# Create given student EVs
studentEV = df.groupby(["stud_id"]).mean()

## 2: Student Class

In [5]:
class Student:
    
    def __init__(self, data):
        self.data = data
        self.means = 0
        self.problem = ""

        
    def updateStudent(self, lastQ):
        self.data = self.data.append(lastQ)
        self.means = self.data.mean()

        
    def nextQ(self):
        # Determines a viable next question
        EVdur = self.means.duration
        EVinc = self.means.incorrect
        EVhint = self.means.hint
        viableProbs = problemDists[(problemDists["durationUpper"] > EVdur)  & (problemDists["hintUpper"] > EVhint) & \
                                   (problemDists["incorrectLower"] > EVinc)]
        problemNum = random.choice(viableProbs.index.values.tolist())
        self.problem = df[df["problem_id"]==problemNum].iloc[0][0].split(':')[-1].split("?")[0] + "?"

## 3: Example Run Through

In [9]:
# Pseudo Code
# studentData = Pretest data
# Pretest data will include entries with "step", "problem_id", "duration", "hint", "incorrect", "correct"
# student = Student(studentData)

# Example
stud = "Stu_fe96fe63d83aa63c4ec667167fc7f1ce"
df = data[["step", "problem_id", "stud_id","duration", "hint", "incorrect", "correct"]]
newdf = df[df["stud_id"] == stud]
studentData = newdf.iloc[0:10,:]

# Initializing a student
student = Student(studentData)

# questionData represents their next action from the interface (we will be passed this)
questionData = newdf.iloc[11:13,:]

# If the person gets the problem right, then send the questionData to the student to update and choose next Q
if questionData.correct.isin([1]).any():
    student.updateStudent(questionData)
    student.nextQ()

# Question that is sent to the interface next:
print(student.problem)

How many units apart are -6 and 4 on the number line?


# What is left to finish model?

- Need to find correct answers
- Need to find images
- Need to figure out how to get through steps
- Need to format answer box to accept all close answers